In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import configparser
import pathlib
import logging
from datetime import datetime
import sys

In [31]:
ls ./..

Archive/   Data/      Logs/      README.md
Config/    DataModel/ Notebook/  SOW/


In [32]:
ls ./../Data

AUS.xlsx  IND.csv   USA.csv


In [33]:
def getSource_File_Path(logger):
    logger.info("[INFO]: Reading configuration file.")
    conffilepath = '/Volumes/E-Study/Github/Data-Engineering-Vaccination-Metrics/Config/configfile.ini'
    configfile = pathlib.Path(conffilepath)
    
    if configfile.exists():
        logger.info("[INFO]: Configuration file: " + str(configfile))
        config_obj = configparser.ConfigParser()
        config_obj.read(configfile)
        try:
            srcfilepath = config_obj['parameters']['filepath']
            logger.info("[INFO]: Source file location: " + srcfilepath)
            return srcfilepath
        except KeyError as e:
            logger.error("[ERROR]: filepath parameter not found in the configfile.ini.")
            return 1
    else:
        logger.error("[ERROR]: configfile.ini is missing.")
        logger.error("[ERROR]: configfile.ini expected location: " + str(conffilepath))
        return 1

In [34]:
def getLog_File_Path(logger):
    conffilepath = '/Volumes/E-Study/Github/Data-Engineering-Vaccination-Metrics/Config/configfile.ini'
    configfile = pathlib.Path(conffilepath)
    if configfile.exists():
        logger.info("[INFO]: Configuration file: " + str(configfile))
        config_obj = configparser.ConfigParser()
        config_obj.read(configfile)
        try:
            logpath = config_obj['parameters']['logpath']
            logger.info("[INFO]: Log file location: " + logpath)
            return logpath
        except KeyError as e:
            logger.error("[ERROR]: logpath parameter not found in the configfile.ini.")
            return 1
    else:
        logger.error("[ERROR]: configfile.ini is missing.")
        return 1

In [35]:
def isValid_File_Type(f):
    if f.endswith('.xlsx') or f.endswith('.XLSX'):
        return True
    elif f.endswith('.csv') or f.endswith('.CSV'):
        return True
    elif f.startswith('.'):
        return True
    else:
        return False

In [36]:
def init_log():
    logger=logging.getLogger()
    logpath = getLog_File_Path(logger)
    logfile = logpath + str(datetime.now().strftime('%Y_%m_%d')) + '.log'
    logging.basicConfig(filename=logfile,
                        format='%(asctime)s %(message)s',
                        filemode='w',
                        force=True)
    return logger

In [37]:
def abort():
    sys.exit()
    

In [38]:
def job_Housekeeping(logger):
    logger.info("[INFO]: Starting the full feed process.")
    
    filepath = getSource_File_Path(logger)
    
    if filepath == 1:
        return 1
    
    files = os.listdir(filepath)
    files = [f for f in files if not f.startswith('.')]
    if len(files) == 0:
        logger.error("[ERROR]: Input files are missing. Please contact the source system.")
        return 1

    logger.info("[INFO]: Validating the file types received from source.")
    for f in files:
        if isValid_File_Type(f):
            logger.info("[INFO]: Valid file type. Filename is: " + f)
        else:
            logger.error("[ERROR]: Invalid file type received from source. Filename is: " + f)
            return 1
    return 0

In [67]:
def datacleansing_IND(logger, fname):
    df = pd.read_csv(fname)
    print(df.head())

In [75]:
df_ind

,ID,Name,DOB,VaccinationType,VaccinationDate,Free or Paid
0,1,Vikas,1998-12-01,XYZ,2022-01-01,F
1,2,Rahul,1982-08-13,ABC,2022-03-05,P
2,3,Sameer,1952-08-13,ABC,2022-02-20,F


In [83]:
df_usa.columns

Index(['ID', 'Name', 'VaccinationType', 'VaccinationDate'], dtype='object')

In [68]:
def datacleansing_USA(logger, fname):
    print(fname)

In [69]:
def datacleansing_AUS(logger, fname):
    print(fname)

In [70]:
def job_Datacleansing(logger):
    logger.info("[INFO]: -----------------------------------")
    logger.info("[INFO]: Starting the data cleansing process.")
    filepath = getSource_File_Path(logger)
    files = os.listdir(filepath)
    files = [f for f in files if not f.startswith('.')]
    for f in files:
        fname = filepath + f
        if f in ['IND.csv', 'IND.CSV']:
            datacleansing_IND(logger, fname)
        elif f in ['USA.csv', 'USA.CSV']:
            datacleansing_USA(logger, fname)
        elif f in ['AUS.xlsx', 'AUS.XLSX']:
            datacleansing_AUS(logger, fname)
        else:
            logger.warning("[WARNING]: New country's data is received. Please check.")
            return 2

In [71]:
logger = init_log()
logger.setLevel(logging.DEBUG)

jb_hkeep_rc = job_Housekeeping(logger)
if (jb_hkeep_rc):
    logger.error("[ERROR]: Aborting the housekeeping job.")
    abort()
else:
    logger.info("[INFO]: Housekeeping job ended successfully.")
    job_Datacleansing(logger)

   ID    Name         DOB VaccinationType VaccinationDate Free or Paid
0   1   Vikas  1998-12-01             XYZ      2022-01-01            F
1   2   Rahul  1982-08-13             ABC      2022-03-05            P
2   3  Sameer  1952-08-13             ABC      2022-02-20            F
/Volumes/E-Study/Github/Data-Engineering-Vaccination-Metrics/Data/AUS.xlsx
/Volumes/E-Study/Github/Data-Engineering-Vaccination-Metrics/Data/USA.csv


In [91]:
df_aus.dtypes

Unique ID                       int64
Patient Name                   object
Vaccine Type                   object
Date of Birth          datetime64[ns]
Date of Vaccination            object
dtype: object

In [84]:
df_aus = pd.read_excel('./../Data/AUS.xlsx')
df_aus.head()

,Unique ID,Patient Name,Vaccine Type,Date of Birth,Date of Vaccination
0,1,Mike,LMN,NaT,2022-05-11 00:00:00
1,2,Jonnathan,XYZ,1997-12-13,2021-13-13
2,3,Cristina,ABC,1998-03-12,2022-03-12 00:00:00


In [24]:
df_aus.isna().sum()

Unique ID              0
Patient Name           0
Vaccine Type           0
Date of Birth          1
Date of Vaccination    0
dtype: int64

In [74]:
df_ind = pd.read_csv('./../Data/IND.csv')
df_ind.head()

,ID,Name,DOB,VaccinationType,VaccinationDate,Free or Paid
0,1,Vikas,1998-12-01,XYZ,2022-01-01,F
1,2,Rahul,1982-08-13,ABC,2022-03-05,P
2,3,Sameer,1952-08-13,ABC,2022-02-20,F


In [26]:
df_ind.isna().sum()

ID                 0
Name               0
DOB                0
VaccinationType    0
VaccinationDate    0
Free or Paid       0
dtype: int64

In [82]:
df_usa = pd.read_csv('./../Data/USA.csv')
df_usa.head()

,ID,Name,VaccinationType,VaccinationDate
0,1,Sam,EFG,6152022
1,2,John,XYZ,1052022
2,3,Mike,ABC,12282021


In [28]:
df_usa.isna().sum()

ID                 0
Name               0
VaccinationType    0
VaccinationDate    0
dtype: int64

In [99]:
df_ind.head()

,ID,Name,DOB,VaccinationType,VaccinationDate,Free or Paid
0,1,Vikas,1998-12-01,XYZ,2022-01-01,F
1,2,Rahul,1982-08-13,ABC,2022-03-05,P
2,3,Sameer,1952-08-13,ABC,2022-02-20,F


In [120]:
df_ind.dtypes

ID                  int64
Name               object
DOB                object
VaccinationType    object
VaccinationDate    object
Free or Paid       object
dtype: object

In [209]:
def IND_convert_Object_to_Date(obj):
    try:
        return datetime. strptime(str(obj), '%Y-%m-%d')
    except:
        return pd.to_datetime('2020-01-01')
    

In [243]:
def USA_convert_Object_to_Date(obj):
    try:
        return datetime. strptime(str(obj).strip(), '%m%d%Y')
    except:
        return pd.to_datetime('2020-01-01')

In [265]:
def AUS_convert_Object_to_Date(obj):
    try:
        return datetime. strptime(str(obj).strip(), '%Y-%m-%d')
    except:
        try:
            return datetime.strptime(str(obj).strip(), '%Y-%m-%d %H:%M:%S')
        except:
            return pd.to_datetime('1920-01-01')

In [266]:
AUS_convert_Object_to_Date('1-1-')

Timestamp('1920-01-01 00:00:00')

In [267]:
df_aus['Date of Vaccination'].apply(lambda x: AUS_convert_Object_to_Date(x))

0   2022-05-11
1   1920-01-01
2   2022-03-12
Name: Date of Vaccination, dtype: datetime64[ns]

In [233]:
df_usa['VaccinationDate'] = df_usa['VaccinationDate'].astype(str).apply(lambda x: x.strip())

In [237]:
df_usa

,ID,Name,VaccinationType,VaccinationDate
0,1,Sam,EFG,6152022
1,2,John,XYZ,1052022
2,3,Mike,ABC,12282021


In [247]:
df_aus.dtypes

Unique ID                       int64
Patient Name                   object
Vaccine Type                   object
Date of Birth          datetime64[ns]
Date of Vaccination            object
dtype: object

In [98]:
df_aus.head()

,Unique ID,Patient Name,Vaccine Type,Date of Birth,Date of Vaccination
0,1,Mike,LMN,NaT,2022-05-11 00:00:00
1,2,Jonnathan,XYZ,1997-12-13,2021-13-13
2,3,Cristina,ABC,1998-03-12,2022-03-12 00:00:00


In [112]:
df_aus.loc[df_aus['Date of Birth'].isna()].shape[0]

1

In [119]:
df_aus.loc[~df_aus.isna().any(axis=1)]

,Unique ID,Patient Name,Vaccine Type,Date of Birth,Date of Vaccination
1,2,Jonnathan,XYZ,1997-12-13,2021-13-13
2,3,Cristina,ABC,1998-03-12,2022-03-12 00:00:00


In [269]:
os.path.splitext('AUS.xlsx')[0]

'AUS'

In [273]:
df_i = pd.read_csv('../Staging/IND.csv')
df_u = pd.read_csv('../Staging/USA.csv')
df_a = pd.read_csv('../Staging/AUS.csv')

In [280]:
df_all = pd.concat([df_i, df_u, df_a], ignore_index=True, sort=False)

In [281]:
df_all

,ID,Country,Name,VaccinationType,VaccinationDate
0,1,IND,Vikas,XYZ,2022-01-01
1,2,IND,Rahul,ABC,2022-03-05
2,3,IND,Sameer,ABC,2022-02-20
3,1,USA,Sam,EFG,2022-06-15
4,2,USA,John,XYZ,2022-10-05
5,3,USA,Mike,ABC,2021-12-28
6,1,AUS,Mike,LMN,2022-05-11
7,2,AUS,Jonnathan,XYZ,1920-01-01
8,3,AUS,Cristina,ABC,2022-03-12


In [284]:
df_all.groupby(['Country','VaccinationType']).agg(
        VaccinatedCount = pd.NamedAgg('ID', 'count')
    ).reset_index()

,Country,VaccinationType,VaccinatedCount
0,AUS,ABC,1
1,AUS,LMN,1
2,AUS,XYZ,1
3,IND,ABC,2
4,IND,XYZ,1
5,USA,ABC,1
6,USA,EFG,1
7,USA,XYZ,1


In [286]:
metric2 = df_all.groupby(['Country']).agg(
        VaccinatedCount = pd.NamedAgg('ID', 'count')
    ).reset_index()

In [292]:
x = {'Country': ['IND', 'USA', 'AUS'],
     'Population':[1417422253, 33190000, 25700000]}
population = pd.DataFrame(x)
population

,Country,Population
0,IND,1417422253
1,USA,33190000
2,AUS,25700000
